In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [2]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

In [3]:
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    human_input=True,
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    context=[
        brainstorm_task,
    ],
    output_file="selection_task.md",
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="competitors_task.md",
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="inspiration_task.md",
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video. The results are always provided in Korean.",
    agent=writer,
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="script_task.md",
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        industry="자동차",
    ),
)

result

 [DEBUG]: == Working Agent: Senior Marketer
 [INFO]: == Starting Task: Come up with 5 video ideas for a Youtube channel in the 자동차 industry


> Entering new CrewAgentExecutor chain...
To come up with viral video ideas for a YouTube channel in the 자동차 (automotive) industry, it's important to understand current trends, popular content types, and what engages a young audience. I will start by gathering insights into the latest trends and popular content in the automotive industry from the Senior Researcher. This will help inform the video ideas.

Action: Ask question to co-worker
Action Input: {"question": "What are the latest trends and popular content types in the automotive industry that engage a young audience?", "context": "We are developing video ideas for a YouTube channel in the automotive industry, aiming to make content that resonates with a young audience. Understanding the current trends and what types of content are popular will help in creating engaging and potentially viral

'**제목: 전기차와 함께하는 하루: 일상의 모습**\n\n**소개:**\n- 한적한 도시 거리의 일출 장면으로 시작하여 차량들이 지나가는 모습을 보여줍니다. 카메라는 드라이브웨이에 주차된 세련되고 현대적인 전기차(EV)에 초점을 맞춥니다.\n- 나레이션: "전기차 소유자의 하루를 함께합니다. 오늘은 아침 출근길부터 저녁 휴식까지, 전기차와 함께하는 일상의 모습을 탐험해보겠습니다."\n\n**아침 루틴:**\n- 주인이 자택 충전소에서 EV를 분리하는 모습을 보여줍니다.\n- 나레이션: "많은 EV 소유자에게 하루는 완전히 충전된 차와 함께 시작됩니다. 집에서 충전할 수 있는 편리함 덕분에 더 이상 주유소에 들를 필요가 없습니다. 밤새 충전하면 출발 준비 완료!"\n\n**출근길:**\n- 주인이 도시를 운전하는 모습을 촬영하며 부드럽고 조용한 주행을 강조합니다.\n- 나레이션: "전기차는 조용하고 부드러운 주행 경험을 제공하여 일상적인 통근을 더욱 즐겁게 만듭니다. 즉각적인 토크 덕분에 가속은 빠르고 짜릿합니다."\n\n**환경적 이점:**\n- 가솔린 차량이 배기가스를 배출하는 장면과 EV가 깨끗하게 주행하는 장면을 분할 화면으로 보여줍니다.\n- 나레이션: "전기차 운전의 가장 큰 장점 중 하나는 환경적 이점입니다. EV는 배기가스가 전혀 없기 때문에 깨끗한 공기와 건강한 지구를 만드는 데 기여합니다."\n\n**비용 절감:**\n- 전기 요금과 휘발유 비용을 비교하는 그래픽을 보여줍니다.\n- 나레이션: "EV 소유는 상당한 비용 절감을 의미할 수 있습니다. 전기차 운행 비용은 마일당 약 4센트이며, 오일 교환이나 점화 플러그 교체가 필요 없기 때문에 유지보수도 최소화됩니다."\n\n**점심 시간 인사이트:**\n- 충전소에서 EV 소유자와의 인터뷰를 통해 그들의 생각을 공유합니다.\n- 소유자: "내 차를 운전하는 것이 얼마나 저렴한지 정말 좋아요. 집에서 충전하는 것은 매우 저렴하고, 올바른 설정을 통해 그리드에 에너지를 다시 판매할 수도 있습니다.